In [16]:
import os
from llama_index.llms.perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv()

pplx_api_key = os.getenv("PERPLEXITY_API_KEY")

llm = Perplexity(
    api_key=pplx_api_key, model="mistral-7b-instruct", temperature=0.5
)

In [17]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [18]:
from llama_index.core.indices.knowledge_graph.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [5]:
from llama_index.core.llms import ChatMessage

messages_dict = [
    {"role": "system", "content": "Be precise and concise."},
    {"role": "user", "content": "tell me about harry potter"},
]
messages = [ChatMessage(**msg) for msg in messages_dict]

response = llm.chat(messages)
print(response)

assistant: Harry Potter is a series of seven fantasy novels written by British author J.K. Rowling. The books follow Harry Potter, an orphaned boy who discovers he is a wizard and is invited to attend Hogwarts School of Witchcraft and Wizardry. At Hogwarts, Harry learns magic, makes friends, and faces numerous challenges, including battles against the dark wizard Lord Voldemort. The series is known for its complex and intricate plot, engaging characters, and richly detailed magical world. The books have been translated into over 80 languages and have sold over 500 million copies worldwide. The Harry Potter series also inspired a successful film series, a play, and various merchandise.
